In [2]:
from wsgiref import headers
import requests
from bs4 import BeautifulSoup 
import pandas as pd


In [3]:

yes = pd.DataFrame()
a=1
for i in range(a,13):
    URL = 'https://sgodds.com/football/results-past-odds/page/{}'.format(a)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'}
    sayfa = requests.get(URL, headers = headers)
    icerik = BeautifulSoup(sayfa.content,"lxml")
    a +=1
    takimAdlari =icerik.find_all("div", attrs = {"class":"row border-bottom m-0"})
    strong = icerik.find_all("strong")
    takimlar = []
    skorlar = []
    ligler = []
    
    for takimAdi in takimAdlari:
        takimlar.append(takimAdi.a.text)
        ligler.append(takimAdi.div.text)
        skorlar.append(takimAdi.find("div", attrs = {"row"}).text)  
    b = 0
    for i in skorlar:
        if len(i) != 6:
            skorlar[b] = "0-00-0"
        if i == "\xa0 -\xa0 -":
            skorlar[b] = "0-00-0" 
        b+=1
    macSonucu = []    
    ilkYari = []
    for ilk in skorlar:
        ilkYari.append(ilk[0:3])
    for ms in skorlar:
        macSonucu.append(ms[3:len(ms)])
    altUst = []
    for i in skorlar:
        if (int(i[0]) + int(i[2]) + int(i[3]) + int(i[5])) > 2.5:
            altUst.append("UST")
        else:
            altUst.append("ALT")
    evSahibiiy = []
    dePiy = []
    evSahibiMs = []
    depMs = []
    for i in ilkYari:
        evSahibiiy.append(int(i[0]))

    for i in ilkYari:
        dePiy.append(int(i[2]))
        
    for i in macSonucu:
        evSahibiMs.append(int(i[0]))

    for i in macSonucu:
        depMs.append(int(i[2]))     
    oranlar = []      
    for i in strong:
        oranlar.append(i.text)
        
    yeniOranlar = oranlar[2:-6]
    N = 3
    subList = [yeniOranlar[n:n+N] for n in range(0, len(yeniOranlar), N)]  
    evSahibiOran = []
    beraberlikOran= []
    deplasmanOran= []
    for i in subList:
        evSahibiOran.append(float(i[0]))
    for i in subList:
        beraberlikOran.append(float(i[1]))
    for i in subList:
        deplasmanOran.append(float(i[2]))
    evSahibi = []
    deplasman = []
    for i in takimlar:
        evSahibi.append(i.split("vs")[0])
        
    for i in takimlar:
        deplasman.append(i.split("vs")[1])
        
    dicti = {'League' : ligler , 'Home_Team':evSahibi,'Away_Team':deplasman,'Home_FH':evSahibiiy,"Away_FH":dePiy,"Home_Result":evSahibiMs,"Away_Result":depMs,"Odds_Home":evSahibiOran,"Odds_Draw":beraberlikOran,"Odds_Away":deplasmanOran,"Alt-Ust":altUst}
    
    daf = pd.DataFrame(dicti)
    
    yes = pd.concat([yes,daf])


                  League     Home_Team         Away_Team  Home_FH  Away_FH  \
0         Mexican League   Club Atlas             Necaxa        1        0   
1         Mexican League      Pachuca        Tigres UANL        0        0   
2              UE Europe     West Ham               Lyon        0        0   
3              UE Europe        Braga    Glasgow Rangers        1        0   
4              UE Europe  E Frankfurt          Barcelona        0        0   
..                   ...           ...               ...      ...      ...   
151  African Nations Cup        Gabon              Ghana        0        1   
152  African Nations Cup       Malawi           Zimbabwe        1        1   
153  African Nations Cup      Morocco            Comoros        1        0   
154      Indian S League       FC Goa      NorthEast Utd        1        1   
155  African Nations Cup      Senegal             Guinea        0        0   

     Home_Result  Away_Result  Odds_Home  Odds_Draw  Odds_Away 

In [1]:
prem = yes[yes['League'] == "English Premier"]
prem.reset_index(drop=True, inplace=True)
prem.to_csv("Premier_league.csv",index=False)
prem

NameError: name 'yes' is not defined